Project goals:
- Scrape laptop data from Takealot and Amazon sites to find arbitrage opportunities (for laptops between R10000 and R20000).
- Process must be automated.
- Output must be dataframe of laptop brand, name, price, stock-status if available (in/out of stock), and link to laptop info for both sites. Laptops will get matched on similarity based on a selected matching ratio (90%), which will also be included in the final output, to compensate for slight naming differences on both sites.
- Disclaimer: this project was not designed for practical use.

The following is the relevant code to produce this output (should take about 10 minutes to run - need to investigate how I can significantly reduce it):

In [1]:
# Import relevant libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import requests
import re
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException, StaleElementReferenceException
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Starting WebDriver Session
# If you don't want the page (GUI - Graphical User Interface) to show up, you can run...
# the following code (recommended when system is fully automated - will cut down on...
# computational expenses):

options = Options()
options.add_argument("--headless") # Run without popping up GUI
# set options parameter in 'driver' object to options object during instantiation

# Initializing firefox webdriver
driver = webdriver.Firefox(options)

# DEFINING A FUNCTION THAT WILL OUTPUT TABLE WITH TAKEALOT LAPTOP DATA

def get_takealot_laptop():
    
    # open the desired URL
    driver.get("https://www.takealot.com")
    
    # wait for URL to load
    time.sleep(5)
    
    # Interacting with input field using By module and find_element() method
    #input_field = driver.find_element(By.NAME, "search") # Finding search box
    input_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "search"))
    )
    input_field.send_keys('laptop') # Writing 'laptop' as search input in search box
    
    # Submitting inputted keys 
    input_field.submit()
    
    # wait for laptop options to load
    time.sleep(5)
    
    # While loop to load all laptops by repeatedly clicking the 'Load More' button until all the options are laoded
    while True:
        # try block will attempt to find and click 'Load More' button while it is findable or not stale
        try:
            # Count items on page before clicking 'Load More'
            current_count = len(driver.find_elements(By.CSS_SELECTOR, "div.search-product"))
            
            # Finding the 'Load More' button
            #load_more_button = driver.find_element(By.XPATH, "//button[text()='Load More']")
            load_more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//button[text()='Load More']"))
            )
            # Scroll the element into view
            driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
            # Using ActionChanins() class to simulate/automate actions taken to click on the 'Load More button'
            ActionChains(driver).move_to_element(load_more_button).click(load_more_button).perform()

            # Wait for items on page to increase (signal that page has loaded) before re-iterating loop
            WebDriverWait(driver, 20).until(
                lambda d: len(d.find_elements(By.CSS_SELECTOR, "div.search-product")) > current_count)
                    
        # If 'Load More' button doesn't show up within the exception's default time, break
        # This is a check to see if 'Load More' button is actually gone if item count doesn't change
        except TimeoutException as e:
            if not driver.find_elements(By.XPATH, "//button[text()='Load More']"):
                break
        else:
            continue
            
    # Extract html of current page and store it in an object
    html_takealot = driver.page_source
    
    # close currently opened tab
    #driver.close()
            
    # Prettyfying the html
    soup_takealot = BeautifulSoup(html_takealot, 'lxml')
    
    # CREATING A FOR-LOOP THAT WILL EXTRACT LAPTOP DETAILS WITHIN A PRICE RANGE OF R10000 AND R20000, THEN...
    # STORE THEM AS A DATAFRAME

    # initialize an empty list to store the Amazon laptop data
    takealot_data = []

    # extracting details for all laptops
    laptops_details = soup_takealot.find_all('div', class_ = 'search-product grid')
    # grid-y gap-1 product-card-module_product-details-container_3ntEA

    # Define price range
    min_price = 10000
    max_price = 20000

    # for-loop
    for laptop_deets in laptops_details:
        # extracting laptop name from 'laptop_details' object
        # .text if you just want the text
        name = laptop_deets.find('h4', class_ = 'product-card-module_product-title_16xh8')
        name = name.text if name else "None"
        # extracting laptop brand from 'laptop_details' object
        brand = laptop_deets.find('a', href=re.compile(r'Brand'))
        brand = brand.text if brand else "None"
        # extracting text and numerical (for subsetting) laptop price from 'laptop_details' object
        price = laptop_deets.find('span', class_ = 'currency plus currency-module_currency_29IIm')
        price_text = price.text if price else "None"
        if price:
            price_stripped = re.sub(r'\D', '', price.text)
            price_numerical = int(price_stripped)
        # extracting laptop stock status from 'laptop_details' object
        stock_status = laptop_deets.find('div', class_ = 'grid-y gap-1 in-stock-indicator-module_in-stock-indicator_3kr9C')
        stock_status = stock_status.text if stock_status and stock_status.text else "No stock"
        # extracting link for laptop
        link = laptop_deets.find('a', class_ = 'product-card-module_link-underlay_3sfaA')
        link = 'https://www.takealot.com' + link.get('href') if link else "No link"

        # Filter laptops within price range and create dictionary output
        if price and min_price <= price_numerical <= max_price:
            #print(f'''
            #Name: {name}
            #Brand: {brand}
            #Price: {price_text}
            #Stock status: {stock_status}
            #Link: {link}
            #''')
            takealot_data.append({
                'Brand': brand,
                'Name': name,
                'Price': price_text,
                'Stock status': stock_status,
                'Link': link
            })

            # print line space as separator between laptop details
            #print('')

    # convert list of dictionaries into a Pandas dataframe
    takealot_df = pd.DataFrame(takealot_data)

    # return the dataframe
    return takealot_df

# run function (will take some time)
takealot_laptop = get_takealot_laptop()

# DEFINING A FUNCTION THAT WILL OUTPUT TABLE WITH AMAZON LAPTOP DATA

def get_amazon_laptop():
    
    # open the desired URL
    driver.get("https://www.amazon.co.za")
    
    # wait for URL to load
    time.sleep(5)
    
    # Interacting with input field using By module and find_element() method
    # Finding search box
    #input_field = driver.find_element(By.NAME, "field-keywords")
    input_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "field-keywords"))
    )
    # Writing 'laptop' as search input in search box
    input_field.send_keys('laptop')
    
    # Submitting inputted keys 
    input_field.submit()
    
    # wait for URL to load
    time.sleep(5)
    
    # Initialize empty object where Amazon html will be contained
    html_amazon = []

    # While loop to iterate through pages by pressing the 'next' button then scraping html on current page
    while True:
        try:
            # Scrape the current page's html and add it to the 'html_amazon' object with a page delimiter
            html_amazon.append(driver.page_source)

            # Get current page number
            current_page = driver.find_element(By.CSS_SELECTOR, "span.s-pagination-item.s-pagination-selected").text

            # waiting 10 seconds maximum for next button to become clickable and assigning it to an object once found
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 's-pagination-next')]"))
            )
            # If 'next_button' is disabled, break the loop
            if "disabled" in next_button.get_attribute("class") or next_button.get_attribute("aria-disabled") == "true":
                #print("No more pages")
                break
            # Scrolling the next button into view (specifically center view)
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
            # Using ActionChanins() class to simulate/automate actions taken to click on the 'Load More button'
            ActionChains(driver).move_to_element(next_button).click(next_button).perform()

            # Wait until page number changes
            WebDriverWait(driver, 20).until(
                lambda d: d.find_element(By.CSS_SELECTOR, "span.s-pagination-item.s-pagination-selected").text != current_page)

            # Wait until results are present
            WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-component-type='s-search-result']")))

        # If 'Next' button doesn't show up within the exception's default time, break
        except TimeoutException as e:
                break
            
    # combine all html pages into a single string
    combined_html = "<div>" + "".join(html_amazon) + "</div>"
    
    # quit WebDriver session
    driver.quit()
    
    # Prettyfying the html
    soup_amazon = BeautifulSoup(combined_html, 'lxml')
    
    # CREATING A FOR-LOOP THAT WILL EXTRACT LAPTOP DETAILS WITHIN A PRICE RANGE OF R10000 AND R20000, THEN...
    # STORE THEM AS A DATAFRAME

    # initialize an empty list to store the Amazon laptop data
    amazon_data = []

    # extracting details for all laptops
    laptops_details = soup_amazon.find_all('div', class_ = 'a-section a-spacing-small puis-padding-left-small puis-padding-right-small')
    # grid-y gap-1 product-card-module_product-details-container_3ntEA

    # Define price range
    min_price = 10000
    max_price = 20000

    # for-loop
    for laptop_deets in laptops_details:
        # extracting laptop name from 'laptop_details' object
        # .text if you just want the text
        name = laptop_deets.find('h2', class_ = 'a-size-base-plus a-spacing-none a-color-base a-text-normal')
        name = name.text if name else "None"
        # extracting laptop brand from 'laptop_details' object
        #~brand = laptop_deets.find('a', href=re.compile(r'Brand'))
        #~brand = brand.text if brand else "None"
        # extracting text and numerical (for subsetting) laptop price from 'laptop_details' object
        price = laptop_deets.find('span', class_ = 'a-price-whole')
        price_text = 'R ' + price.text.replace('.', '') if price else "None"
        if price:
            price_stripped = re.sub(r'\D', '', price.text)
            price_numerical = int(price_stripped)
        # printing laptop stock status 
        stock_status = "click link to view"
        # extracting link for laptop
        link = laptop_deets.find('a', class_ = 'a-link-normal s-line-clamp-4 s-link-style a-text-normal')
        link = 'https://www.amazon.co.za' + link.get('href') if link else "No link"

        # filter laptops within price range and create dictionary output
        if price and min_price <= price_numerical <= max_price:
            #print(f'''
            #Name: {name}
            #Price: {price_text}
            #Stock status: {stock_status}
            #Link: {link}
            #''')
            amazon_data.append({
                'Name': name,
                'Price': price_text,
                'Stock status': stock_status,
                'Link': link
            })

            # print line space as separator between laptop details
            #print('')

    # convert list of dictionaries into a Pandas dataframe
    amazon_df = pd.DataFrame(amazon_data)

    # display the dataframe
    return amazon_df   

# run function (will take a minute or two)
amazon_laptop = get_amazon_laptop()

# creating a column with normalized names for takealot dataframe
takealot_laptop['Name Normalized'] = takealot_laptop['Name'].str.lower().str.strip().str.replace("|", "")

# creating a column with normalized names for amazon dataframe
amazon_laptop['Name Normalized'] = amazon_laptop['Name'].str.lower().str.strip().str.replace("|", "")

# Perform a merge on 'Name Normalized'
linked_data = pd.merge(takealot_laptop, amazon_laptop, on='Name Normalized', suffixes=('_takealot', '_amazon'))

# FUZZY MATCHING

# create a list of names from Takealot data's 'Name Normalized' column as a 'matching dictionary'
takealot_names = takealot_laptop['Name Normalized'].tolist()

# create fuzzy matching function
def match_names(row, name_dict, threshold):
    match = process.extractOne(row['Name Normalized'], name_dict, scorer=fuzz.token_sort_ratio)
    if match and match[1] > threshold: # check if similarity meets threshold
        return match[0], match[1] # return matched name and similarity if condition is met
    else:
        return None, None # return 'None' if condition is not met

# applying fuzzy matching function to Amazon's 'Normalized Name' column using apply() method on row-by-row basis
amazon_laptop[["Match", "Match Similarity"]] = amazon_laptop.apply(
    lambda row: pd.Series(match_names(row, name_dict=takealot_names, threshold=90)), 
    axis=1
)

# Merge the dataframes
# pd.merge() does exact matching; the match_names function gives approximate matches keys that exact matches would have.
fuzzy_linked_data = pd.merge(
    takealot_laptop,
    amazon_laptop,
    left_on='Name Normalized',
    right_on='Match',
    suffixes=('_takealot', '_amazon')
)

# remove unnecessary columns for the aesthetic
fuzzy_linked_data.drop(columns=['Name Normalized_takealot',
                                'Name Normalized_amazon',
                                'Match'], inplace=True
                      )

fuzzy_linked_data

,Brand,Name_takealot,Price_takealot,Stock status_takealot,Link_takealot,Name_amazon,Price_amazon,Stock status_amazon,Link_amazon,Match Similarity
0,Huawei,Huawei MateBook D16 Intel Core i5-13420H 16GB ...,"R 16,999",In stockJHBThis item can be shipped from Johan...,https://www.takealot.com/huawei-matebook-d16-i...,Huawei MateBook D16 Intel Core i5-13420H 16GB ...,"R 14 999,",click link to view,https://www.amazon.co.za/Huawei-MateBook-Intel...,100.0
1,HP,HP 15S Core I7-1355U 16GB 512GB SSD Windows 11...,"R 14,899",No stock,https://www.takealot.com/hp-15s-core-i7-1355u-...,HP 15S Core I7-1355U 16GB 512GB SSD Windows 11...,"R 14 899,",click link to view,https://www.amazon.co.za/HP-I7-1355U-512GB-Win...,100.0
2,Dell,DELL INSPIRON 3520 | i5 12th gen | 32GB | 512G...,"R 15,299",In stockCPTThis item can be shipped from Cape ...,https://www.takealot.com/dell-inspiron-3520-i5...,DELL INSPIRON 3520 | i5 12th gen | 32GB | 512G...,"R 15 299,",click link to view,https://www.amazon.co.za/DELL-INSPIRON-3520-51...,100.0
3,ASUS,ASUS Vivobook 16 13th gen i5 | 24GB | 512GB NV...,"R 15,299",Ships in 4 - 6 work days,https://www.takealot.com/asus-vivobook-16-13th...,ASUS Vivobook 16 13th gen i5 | 24GB | 512GB NV...,"R 13 999,",click link to view,https://www.amazon.co.za/ASUS-Vivobook-13th-51...,100.0
4,ASUS,ASUS Vivobook Intel Core i7-13620H 8GB 512GB S...,"R 13,499",In stockCPTThis item can be shipped from Cape ...,https://www.takealot.com/asus-vivobook-intel-c...,ASUS Vivobook Intel® Core™ i7-13620H 8GB RAM 5...,"R 13 499,",click link to view,https://www.amazon.co.za/ASUS-Vivobook-Intel%C...,94.0
5,MSI,MSI Thin 15 i5-12450H 32GB 512GB SSD RTX 2050 ...,"R 16,499",In stockCPTThis item can be shipped from Cape ...,https://www.takealot.com/msi-thin-15-i5-12450h...,MSI Thin 15 i5-12450H 32GB 512GB SSD RTX 2050 ...,"R 15 999,",click link to view,https://www.amazon.co.za/MSI-i5-12450H-512GB-G...,100.0


The following code saves the dataframe as an Excel file if required (PC local path must be specified):

In [2]:
# turn final arbitrage dataframe to Excel file (enter desired path + file-save-name in 'pathsave' object)
pathsave = 'C:\\Users\\mufar\\Downloads\\Laptop Arbitrage.xlsx'
fuzzy_linked_data.to_excel(pathsave, index=False)